In [2]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
import json
warnings.filterwarnings('ignore')

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [9]:
repoName = input("input repo name:")

input repo name: numpy


In [13]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
            SELECT
                repo.repo_id,
                repo.repo_name,
                date_trunc( 'month', message.msg_timestamp ) AS message_month,
                message.cntrb_id, 
                AVG ( message_analysis.sentiment_score ) AS average_sentiment
            FROM
                repo, message_analysis, message, issue_message_ref, issues 
                where message_analysis.msg_id = message.msg_id 
                and 
                issue_message_ref.issue_id = issues.issue_id  
                AND
                issue_message_ref.msg_id = message.msg_id 
                AND
                issues.repo_id = repo.repo_id 
                AND
                repo.repo_name = '""" + repoName + """'
            GROUP BY
                repo.repo_id,
                repo.repo_name,
                message.msg_timestamp, 
                message.cntrb_id 
            ORDER BY
                average_sentiment DESC,
                repo_id,
                message_month DESC;
    """)

repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes

,repo_id,repo_name,message_month,cntrb_id,average_sentiment
0,26112,numpy,2019-05-01,279376,0.997719
1,26112,numpy,2020-05-01,411244,0.996709
2,26112,numpy,2019-01-01,321728,0.996535
3,26112,numpy,2017-11-01,290261,0.995447
4,26112,numpy,2017-03-01,291819,0.995317
...,...,...,...,...,...
91394,26112,numpy,2012-10-01,325978,-0.975844
91395,26112,numpy,2012-12-01,290000,-0.976626
91396,26112,numpy,2012-12-01,395963,-0.976634
91397,26112,numpy,2019-06-01,392504,-0.978636


repo_id                       int64
repo_name                    object
message_month        datetime64[ns]
cntrb_id                      int64
average_sentiment           float64
dtype: object